<a href="https://colab.research.google.com/github/richardcmg7/deep_learning_base/blob/main/Deep_Learning_streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Ejemplo de uso de Modelo Preentrenado en Google Colab usando Python-StreamLite


### 1. Probamos con un modelo que pueda dar solución al problema.  

VGG 19 es un modelo de 19 capas entrenado con un dataset de mas de 1 millón de imagenes [ImageNet](http://www.image-net.org) 

In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19

In [ ]:
classifier = VGG19(
    include_top=True,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation='softmax'
)

574726144/574710816 [==============================] - 4s 0us/step


### 2. Cargamos el modelo al formato adecuado. 

In [ ]:
classifier.save("image_classification.hdf5")

In [ ]:
classifier.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

### 3. Habilitamos el framework para la aplicación

In [ ]:
!pip install -q streamlit

     |████████████████████████████████| 9.1 MB 10.3 MB/s 
     |████████████████████████████████| 78 kB 5.5 MB/s 
     |████████████████████████████████| 181 kB 56.2 MB/s 
     |████████████████████████████████| 4.7 MB 47.6 MB/s 
     |████████████████████████████████| 235 kB 36.9 MB/s 
     |████████████████████████████████| 164 kB 49.5 MB/s 
     |████████████████████████████████| 63 kB 904 kB/s 
     |████████████████████████████████| 51 kB 5.8 MB/s 


### 4. Creamos la Aplicación utilizando Streamlit

In [ ]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
from tensorflow.keras.applications.imagenet_utils import preprocess_input, decode_predictions

@st.cache(allow_output_mutation=True)
def load_model():
  model=tf.keras.models.load_model('/content/image_classification.hdf5')
  return model
with st.spinner('Model is being loaded..'):
  model=load_model()

st.write("""
         # Image Classification
         """
         )

file = st.file_uploader("Upload the image to be classified \U0001F447", type=["jpg", "png"])
import cv2
from PIL import Image, ImageOps
import numpy as np
st.set_option('deprecation.showfileUploaderEncoding', False)
def upload_predict(upload_image, model):
    
        size = (180,180)    
        image = ImageOps.fit(upload_image, size, Image.ANTIALIAS)
        image = np.asarray(image)
        img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        img_resize = cv2.resize(img, dsize=(224, 224),interpolation=cv2.INTER_CUBIC)
        
        img_reshape = img_resize[np.newaxis,...]
    
        prediction = model.predict(img_reshape)
        pred_class=decode_predictions(prediction,top=1)
        
        return pred_class
if file is None:
    st.text("Please upload an image file")
else:
    image = Image.open(file)
    st.image(image, use_column_width=True)
    predictions = upload_predict(image, model)
    image_class = str(predictions[0][0][1])
    score=np.round(predictions[0][0][2],5) 
    st.write("The image is classified as",image_class)
    st.write("The similarity score is approximately",score)
    print("The image is classified as ",image_class, "with a similarity score of",score)

Writing app.py


### 5. Lanzamos la aplicación y utilizamos localtunel para permitir el acceso externo. 

In [ ]:
! streamlit run app.py & npx localtunnel --port 8501

2022-09-15 15:37:32.049 INFO    numexpr.utils: NumExpr defaulting to 2 threads.

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.247.82.148:8501

npx: installed 22 in 5.027s
your url is: https://blue-tools-mate-35-247-82-148.loca.lt
2022-09-15 15:38:27.335891: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-09-15 15:38:31.338 No training configuration found in the save file, so the model was *not* compiled. Compile it manually.
49152/35363 [=========================================] - 0s 0us/step
The image is classified as  seat_belt with a similarity score of 0.06913
The image is classified as  crib with a similarity score of 0.77607
The image is classified as  remote_control with a similarity score of 0.49956


In [ ]:
## Fin del Ejemplo

## Fin del Ejemplo
